# Advanced Ollama Features Tutorial 🚀

Welcome to the advanced Ollama tutorial! In this notebook, we'll explore some of the more powerful features of Ollama, including:

1. 🛠️ **Tool Use** - Making AI use custom functions
2. 🖼️ **Multimodal Chat** - Talking about images with AI
3. 🎨 **Multimodal Generate** - Creating content based on images
4. 📊 **Structured Outputs** - Getting organized data from AI
5. 🖼️ **Structured Image Analysis** - Getting structured data about images

Let's dive into these exciting features! 🌟

## Setup 🛠️

First, let's install the packages we'll need for this advanced tutorial:
- `ollama`: For AI model interactions
- `pydantic`: For data validation and structured outputs
- `httpx`: For making HTTP requests (used in multimodal examples)

Let's install these packages:


In [ ]:
%pip install ollama pydantic httpx


## 1. Tool Use: Making AI Use Custom Functions 🛠️

One of the coolest features of advanced AI models is their ability to use tools - custom functions that we define. This allows the AI to perform actual calculations or actions.

In this example, we'll create two simple math functions and let the AI use them to solve problems. Here's how it works:

1. We define some functions (tools) that the AI can use
2. We tell the AI about these tools
3. The AI decides when and how to use them

Let's try it out!


In [ ]:
from ollama import ChatResponse, chat

# Define our tool functions
def add_two_numbers(a: int, b: int) -> int:
    """
    Add two numbers together
    
    Args:
        a (int): First number
        b (int): Second number
    
    Returns:
        int: The sum of the two numbers
    """
    return int(a) + int(b)

def subtract_two_numbers(a: int, b: int) -> int:
    """
    Subtract the second number from the first
    
    Args:
        a (int): Number to subtract from
        b (int): Number to subtract
    
    Returns:
        int: The difference between the numbers
    """
    return int(a) - int(b)

# Define the tools in the format Ollama expects
subtract_tool = {
    'type': 'function',
    'function': {
        'name': 'subtract_two_numbers',
        'description': 'Subtract two numbers',
        'parameters': {
            'type': 'object',
            'required': ['a', 'b'],
            'properties': {
                'a': {'type': 'integer', 'description': 'The first number'},
                'b': {'type': 'integer', 'description': 'The second number'},
            },
        },
    },
}

# Create a dictionary of available functions
available_functions = {
    'add_two_numbers': add_two_numbers,
    'subtract_two_numbers': subtract_two_numbers,
}

# Let's test it with a simple question
messages = [{'role': 'user', 'content': 'What is twenty plus five?'}]
print('Question:', messages[0]['content'])

# Ask the AI and let it use our tools
response: ChatResponse = chat(
    'llama3.1',
    messages=messages,
    tools=[add_two_numbers, subtract_tool],
)

# Handle the AI's response
if response.message.tool_calls:
    # The AI might make multiple tool calls
    for tool in response.message.tool_calls:
        if function_to_call := available_functions.get(tool.function.name):
            print('\nAI is using function:', tool.function.name)
            print('With arguments:', tool.function.arguments)
            result = function_to_call(**tool.function.arguments)
            print('Result:', result)
            
            # Let the AI explain the result
            messages.append(response.message)
            messages.append({
                'role': 'tool', 
                'content': str(result),
                'name': tool.function.name
            })
            
            final_response = chat('llama3.1', messages=messages)
            print('\nAI explains:', final_response.message.content)
else:
    print('\nThe AI chose to answer without using any tools:', response.message.content)


## 2. Structured Outputs: Getting Organized Data 📊

Sometimes we want the AI to give us information in a very specific format. For example, we might want:
- A list of items with specific properties
- Data that follows a certain structure
- Information organized in a particular way

We can use Pydantic to define exactly what structure we want. Let's try getting information about some friends in a structured way!


In [ ]:
from pydantic import BaseModel
from ollama import chat

# Define what information we want about each friend
class FriendInfo(BaseModel):
    name: str
    age: int
    is_available: bool

# Define the structure for a list of friends
class FriendList(BaseModel):
    friends: list[FriendInfo]

# Let's ask about some friends!
prompt = """
I have two friends. 
The first is Ollama, who is 22 years old and is busy saving the world.
The second is Alonso, who is 23 years old and wants to hang out.
Return a list of friends in JSON format.
"""

# Get structured response from AI
response = chat(
    model='llama3.1',
    messages=[{'role': 'user', 'content': prompt}],
    format=FriendList.model_json_schema(),  # Tell AI what format we want
    options={'temperature': 0}  # Make response more consistent
)

# Convert the response to our structured format
friends_response = FriendList.model_validate_json(response.message.content)

# Let's see what we got!
print("Here's what the AI told us about your friends:\n")
for friend in friends_response.friends:
    status = "is available" if friend.is_available else "is not available"
    print(f"🧑 {friend.name} is {friend.age} years old and {status}")

# We can also see the raw JSON structure
print("\nRaw JSON structure:")
print(friends_response.model_dump_json(indent=2))


## 3. Multimodal Chat: Talking About Images 🖼️

Now let's try something really cool - talking with AI about images! We can show the AI an image and ask questions about it.

For this example, you'll need an image file on your computer. The AI can:
- Describe what's in the image
- Answer questions about the image
- Point out specific details

Let's try it out!


In [ ]:
from pathlib import Path
from ollama import chat

def chat_about_image(image_path: str, question: str):
    """
    Have a chat with AI about an image
    
    Args:
        image_path (str): Path to the image file
        question (str): What you want to ask about the image
    """
    # Convert the path to a Path object and check if it exists
    path = Path(image_path)
    if not path.exists():
        return f"Sorry, couldn't find an image at: {image_path}"
    
    try:
        # Ask the AI about the image
        response = chat(
            model='llama3.2-vision',
            messages=[{
                'role': 'user',
                'content': question,
                'images': [path]
            }]
        )
        return response.message.content
    except Exception as e:
        return f"Oops! Something went wrong: {e}"

# Let's try it out!
# You'll need to put in the path to your image
image_path = input('Enter the path to your image: ')

# Let's ask some questions about the image
questions = [
    "What do you see in this image? Be concise.",
    "What colors are most prominent in this image?",
    "Is there any text in this image? If so, what does it say?"
]

print("\nLet's ask the AI about your image!\n")
for question in questions:
    print(f"Question: {question}")
    answer = chat_about_image(image_path, question)
    print(f"AI's answer: {answer}\n")


## 4. Structured Image Analysis 🖼️📊

Now let's combine what we learned about structured outputs with image analysis! We can ask the AI to analyze an image and give us the information in a specific format.

We'll create a structure that includes:
- A summary of the image
- List of objects detected
- Scene type
- Colors present
- Time of day
- Setting (Indoor/Outdoor)
- Any text found in the image

This is super useful when you want to catalog images or analyze them systematically!


In [ ]:
from pathlib import Path
from typing import Literal
from pydantic import BaseModel
from ollama import chat

# Define what information we want about objects in the image
class Object(BaseModel):
    name: str
    confidence: float
    attributes: str

# Define the complete structure for image analysis
class ImageDescription(BaseModel):
    summary: str
    objects: list[Object]
    scene: str
    colors: list[str]
    time_of_day: Literal['Morning', 'Afternoon', 'Evening', 'Night']
    setting: Literal['Indoor', 'Outdoor', 'Unknown']
    text_content: str | None = None

def analyze_image(image_path: str) -> ImageDescription | str:
    """
    Get a structured analysis of an image
    
    Args:
        image_path (str): Path to the image file
    
    Returns:
        ImageDescription: Structured analysis of the image
        str: Error message if something goes wrong
    """
    path = Path(image_path)
    if not path.exists():
        return f"Sorry, couldn't find an image at: {image_path}"
    
    try:
        # Ask the AI to analyze the image
        response = chat(
            model='llama3.2-vision',
            format=ImageDescription.model_json_schema(),
            messages=[{
                'role': 'user',
                'content': 'Analyze this image and return a detailed JSON description including objects, scene, colors and any text detected. If you cannot determine certain details, leave those fields empty.',
                'images': [path]
            }],
            options={'temperature': 0}
        )
        
        # Convert the response to our structured format
        return ImageDescription.model_validate_json(response.message.content)
    
    except Exception as e:
        return f"Oops! Something went wrong: {e}"

# Let's analyze an image!
image_path = input('Enter the path to your image: ')
analysis = analyze_image(image_path)

if isinstance(analysis, ImageDescription):
    print("\n📸 Image Analysis Results:\n")
    print(f"Summary: {analysis.summary}\n")
    
    print("Objects Detected:")
    for obj in analysis.objects:
        print(f"- {obj.name} (Confidence: {obj.confidence:.2f})")
        print(f"  Attributes: {obj.attributes}")
    
    print(f"\nScene: {analysis.scene}")
    print(f"Colors: {', '.join(analysis.colors)}")
    print(f"Time of Day: {analysis.time_of_day}")
    print(f"Setting: {analysis.setting}")
    
    if analysis.text_content:
        print(f"\nText Detected: {analysis.text_content}")
else:
    print(f"Error: {analysis}")


## 5. Multimodal Generate: Fun with Comics! 🎨

For our final example, let's do something fun - we'll analyze XKCD comics! This example shows how to:
- Fetch images from the web
- Use AI to understand and explain comics
- Stream the AI's response in real-time

This is a great example of combining web requests, image analysis, and streaming responses!


In [ ]:
import random
import sys
import httpx
from ollama import generate

def explain_xkcd_comic(comic_number: int | None = None):
    """
    Fetch and explain an XKCD comic
    
    Args:
        comic_number: Optional specific comic number to explain
    """
    try:
        # Get the latest comic info
        latest = httpx.get('https://xkcd.com/info.0.json')
        latest.raise_for_status()
        
        # Pick a random comic if none specified
        num = comic_number if comic_number else random.randint(1, latest.json().get('num'))
        
        # Get the comic info
        comic = httpx.get(f'https://xkcd.com/{num}/info.0.json')
        comic.raise_for_status()
        comic_data = comic.json()
        
        print(f"XKCD #{comic_data.get('num')}: {comic_data.get('alt')}")
        print(f"Link: https://xkcd.com/{num}")
        print("---")
        
        # Get the image
        image = httpx.get(comic_data.get('img'))
        image.raise_for_status()
        
        # Have the AI explain the comic, streaming the response
        print("AI's explanation:")
        for response in generate(
            'llava',
            'explain this comic:',
            images=[image.content],
            stream=True
        ):
            print(response['response'], end='', flush=True)
        print("\n")
        
    except Exception as e:
        print(f"Oops! Something went wrong: {e}")

# Let's try it with a random comic!
explain_xkcd_comic()

# You can also try a specific comic number:
# explain_xkcd_comic(1234)


## Wrapping Up 🎉

Congratulations! You've learned about some really powerful features of Ollama:

1. 🛠️ **Tool Use**: Making AI use custom functions to perform real actions
2. 📊 **Structured Outputs**: Getting organized, validated data from AI
3. 🖼️ **Multimodal Chat**: Having conversations about images
4. 📸 **Structured Image Analysis**: Getting detailed, organized information about images
5. 🎨 **Multimodal Generate**: Working with images and streaming responses

Things to try next:
- Create your own tools for the AI to use
- Define different structures for the data you want
- Try analyzing different types of images
- Combine these features in creative ways!

Remember:
- Always check if the AI model you want to use is available (`llama3.2-vision` for images)
- Handle errors gracefully as network or model issues can occur
- Save interesting results for later reference

Have fun exploring these advanced features! 🚀
